# TF- IDF(Term Frequency - Inverted Document Frequency)란?

TF-IDF란 1세대 정보검색 기술로 page rank가 등장하기 전 사용되었던 기술이다. 아주 간단한 정보검색을 하는 방식은 각 document에 어떤 word가 있는 지 큰 테이블에 저장하는 방식이였다. 하지만, 이런 경우 table에서 모든 document를 하나씩 검색해서 찾아야 했고, 또 각 테이블마다 모든 단어의 존재 유무 및 개수를 저장해놓고 있어야 했다. 

## Inverted Table
이렇게 되면, 단어의 수가 몇 만, 십만, 백만 개가 되고 document또한 비슷한 개수가 되면, 이 정보를 모두 가지고 있는 table을 만든다는 건 상상이 되지 않는다. 이럴 때 등장한 것이 INVERTED TABLE이다.

Inverted Table은 기준이 문서가 아니라 단어이다. 그래서, 각 단어를 기준으로 그 단어가 속한 document의 index를 linked list로 구현한 것이 inverted Table이다. 

## TF-IDF
하지만, inverted table은 그 단어가 속한 document만 보여줄 뿐, 그 단어와 document가 얼마나 연관성이 있는 지 보여주진 못한다. 그래서, 이 연관성을 보여주기 위해서 나온 것이 tf-idf이다.

![alt text](tf_idf_formula.PNG)

위에서 소문자 t는 우리가 구하고 싶은 term, 소문자 d는 document이다. 대문자 N은 모든 문자의 개수이다. 

### TF (Term Frequency)
TF Vector는 word count vector이다. 즉, 특정한 term 단어가 document에 얼마나 자주 나오는 지를 보여주는 지표이다. 즉, 이 값이 높을 수록 이 단어는 문서에서 중요하다고 볼 수 있다. 

### DF (Document Frequency)
Doucment Frequency, 문서 빈도는 이 term이 얼마나 다른 document에 나왔는 지를 보여주는 지표이다. 이 값의 역수가 idf (inverse document frequency)이다. 

즉, 특정한 term이 특정한 document에 많이 나올수록 (TF가 높을 수록), 반대로 term이 나온 문서의 개수가 적을수록 (N의로 나뉘어지기 때문에 df값이 클수록 값이 작아진다) 그 term과 document사이의 tf-idf값이 커지게 된다!

### Cosine Similarity
그러면 이제 query와 document간의 각도를 활용한 cosine similarity를 사용하여 얼마나 연관이 있는 지를 볼수가 있다. document와 query의 tf-idf값과 query만의 tf-idf값을 구한 후, cosine similarity를 계산할 수 있다. 그런 다음, cosine similarity의 값을 기준으로 가장 높을 5개의 document를 구하다.

![title](cosine_similarity.png)

# Implementation of tf-idf

그러면 이제 tf-idf를 실제 implement해보자. 먼저, 아래와 같은 flow를 가진다. 

![text](flow.png)

### 1. Documents를 index한다. 

### 2. 각 document를 token화 하고 stopwords를 제거하고 stemming해준다. 

### 3. 각 document에서 각 token별로 token이 몇개 있는지를 table로 저장한다.

### 4. 각 token별 어떤 document가 있는 지 inverted index를 구한다

### 5. inverted index를 통해 query를 포함하는 document를 구한다. 

### 6. query와 document 사이의 tf-idf를 구한다. 

### 7. query의 tf-idf를 구한다. 

### 8. 각 query와 document간의 cosine similarity를 구한다. 

### 9. cosine similarity를 기준으로 상위 5개의 document를 구한다. 

# 1. Documents의 index구하기.

In [1]:
import os
import pandas as pd
import numpy as np
import collections
from collections import defaultdict
from collections import OrderedDict

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# 모든 document를 저장할 수 있는 text_set 선언
text_set = [] 

text_dict = defaultdict(list) # 각 text의 내용과 index를 저장하는 defaultdict

# 모든 데이터가 저장되어 있는 곳.
path = 'data/'

for filename in os.listdir(path):
    file = open(os.path.join(path,filename), 'r', encoding='utf-8')
    text = file.readlines() # 내용을 읽는다. 
    filename_int = int(filename.replace(".txt", "")) # 파일 이름에서 .txt를 지우고 index값을 int로 바꾼다. 
    text_dict[filename_int] = text 

In [3]:
len(text_dict)

60

In [4]:
ordered_text_dict = OrderedDict(sorted(text_dict.items())) # index별로 sort한다.

for text in ordered_text_dict: #text_set에 넣는다. 
    text_set.append(ordered_text_dict[text])

In [5]:
text_set

[[' (CNN)If you remember Miley twerking or Kanye\'s mic-snatching, you\'ll know music awards can get chaotic.Like on Tuesday, when singer Taylor Swift took a Twitter dig at hip-hop queen Nicki Minaj and the Internet erupted into a sprawling conversation about race and gender. And then Kim Kardashian got involved. Stay with us.\'Black women are rarely rewarded\'It started when Minaj took to her keyboard to vent after one of her biggest hits wasn\'t nominated for video of the year for the MTV Video Music Awards.Mind you, this isn\'t any old clip. "Anaconda," an uproarious, raunchy twist on pop culture portrayals of curvy black women, received 19.6 million clicks in its first 24 hours, a music video record at the time. So the rapper had a reason to be upset when she was snubbed -- and she made her reasoning clear."If I was a different \'kind\' of artist, Anaconda would be nominated for best choreo and vid of the year as well," said Minaj."Black women influence pop culture so much but are 

# 2. 각 document를 token화 하고 stopwords를 제거하고 stemming해준다.

In [6]:
# stopwords 불러오기.
stop = set(stopwords.words('english'))

# 영어 소문자만 걸러내기 위한 regex
regex = re.compile('^[a-z]*[a-z]$')

vocab_set = set() # 모든 단어의 set
vocab_list = [] # 각 document별로 단어.
porter_stemmer = PorterStemmer() # stemmer

#for i in range(len(text_set)):
for i in range(len(text_set)):
    tokenized_temp = word_tokenize(text_set[i][0].lower()) # 소문자로 만든 뒤 tokenized를 했다. 
    split_tokenized = [] # hypen이나 .으로 연결되어 있는 token들을 포함하여 전체적인 token을 일시적으로 포함하는 list.
        
    for token in tokenized_temp:
        if '-' in token: # '-'으로 연결되어 있으면 나눠준다. 
            split_tokens = token.split('-')
            for split_token in split_tokens:
                split_tokenized.append(split_token)
        elif '.' in token: # '.'으로 연결되어 있으면 나눠준다. 
            split_tokens = token.split('.')
            for split_token in split_tokens:
                split_tokenized.append(split_token)
        else: # 해당되지 않으면 넣는다. 
            split_tokenized.append(token)
    
    
    # stopwords가 아닌것과 알파벳인것만 vocab_list에 넣는다. 
    vocab_list.append([porter_stemmer.stem(token) for token in split_tokenized if token not in stop and regex.match(token)])
    
    
    # 각 vocab을 vocab_set에 넣는다. 
    for vocab in vocab_list[i]:
        vocab_set.add(vocab)

In [7]:
# vocab_list는 각 text마다 들어가 있는 단어 list
# vocab_set은 모든 단어의 집합
print(len(vocab_list), len(vocab_set))

60 4091


# 3. 각 document에서 각 token별로 token이 몇개 있는지를 table로 저장한다.

In [8]:
from collections import defaultdict
import collections

doc_dictionary_list = [] # 각 document별 각 word별로 몇 개 있는 지를 저장하는 dictionary를 모아놓은 list

for i in range(len(vocab_list)):    
    doc_dictionary = defaultdict(int) # 각 document별로 word가 몇 개 있는 지 저장되어 있다. 

    for vocab in vocab_list[i]: # vocab_list에 단어가 몇개가 있는 지 저장 한다. 
        doc_dictionary[vocab] += 1

    for vocab in vocab_set: # 그리고 나머지는 다 0으로 채운다. 
        if doc_dictionary[vocab] != 0:
            pass
    
    od = collections.OrderedDict(sorted(doc_dictionary.items())) # keyword들을 sort한다. 
    
    doc_dictionary_list.append(od) # 마지막으로 lsit에 저장한다. 

# 4. 각 token별 어떤 document가 있는 지 inverted index를 구한다

In [9]:
inverted_index_dic = defaultdict(list) # inverted_index를 포함하는 dictionary

for vocab in vocab_set: # 만약 doc_dictionary에서 그 단어가 1이상 나오면 그 document의 index를 저장한다. 
    index = 0
    for doc_dictionary in doc_dictionary_list:
        if doc_dictionary[vocab] >= 1: # 1이상인 경우 
            inverted_index_dic[vocab].append(index)
    
        index += 1

In [10]:
inverted_index_dic

defaultdict(list,
            {'dancefloor': [3],
             'particularli': [41],
             'london': [39, 41, 42, 46, 47, 51, 52],
             'well': [0,
              1,
              5,
              9,
              11,
              16,
              18,
              24,
              27,
              28,
              31,
              32,
              33,
              34,
              41,
              42,
              47,
              50,
              52,
              57,
              58,
              59],
             'rout': [47],
             'consid': [34, 35, 36, 41, 43, 47, 53],
             'uncontroversi': [9],
             'elli': [26],
             'underlin': [51],
             'impuls': [21, 46],
             'month': [0,
              1,
              3,
              10,
              13,
              15,
              18,
              20,
              23,
              24,
              25,
              27,
              28,
              2

# 5. inverted index를 통해 query를 포함하는 document를 구한다.

In [11]:
with open('query.txt', 'r') as f:
    query = f.readline()

In [12]:
query

'president obama'

In [13]:
query_word = query.split(' ') # query문을 token별로 자른다. 
for i in range(len(query_word)): # 각 query word에 stemming을 해준다. 
    query_word[i] = porter_stemmer.stem(query_word[i])
    
set_presi = set(inverted_index_dic[query_word[0]]) # presi가 있는 document의 index를 구한다. 
set_obama = set(inverted_index_dic[query_word[1]]) # obama가 있는 document의 index를 구한다. 

query_doc_set = set() # 두 query가 동시에 들어가 있는 set을 구한다. 
query_doc_set = set_presi & set_obama

In [14]:
print(query_doc_set)

{36, 6, 40, 41, 43, 46, 47, 48, 49, 50, 53, 54, 57, 58}


# 6. query와 document 사이의 tf-idf를 구한다.

In [15]:
# 각 document에서 query의 tf를 구한다.
# query[0]의 tf와 query[1]의 tf를 각각 구한다.
doc_tf = np.zeros((len(query_doc_set), len(vocab_set))) # doc_tf를 먼저 0으로 채운다. 
query_doc_list = sorted(list(query_doc_set))
vocab_list = sorted(list(vocab_set))

for i in range(len(query_doc_set)):
    for j in range(len(vocab_set)):
        doc_tf[i][j] = doc_dictionary_list[query_doc_list[i]][vocab_list[j]] #각 document에서 각 단어의 개수

In [16]:
doc_tf

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 3., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
query_word_0_index, query_word_1_index = [i for i,t in enumerate(doc_dictionary_list[6]) if t == query_word[0]][0],\
[i for i,t in enumerate(doc_dictionary_list[6]) if t == query_word[1]][0]

In [18]:
query_word_0_index, query_word_1_index

(2739, 2457)

In [19]:
for i in range(len(query_doc_set)):
    print(doc_tf[i][query_word_0_index], doc_tf[i][query_word_1_index])

2.0 4.0
1.0 1.0
3.0 1.0
15.0 2.0
1.0 2.0
2.0 10.0
7.0 10.0
5.0 4.0
9.0 2.0
3.0 6.0
2.0 3.0
9.0 3.0
4.0 1.0
8.0 1.0


In [20]:
# df를 구한다. 
# 각 query[0]과 query[1]이 다른 document에 몇개나 나왔는지 구하고 나머지는 0처리를 한다. 
df = np.zeros((len(vocab_set)))

for i in range(len(df)):
    df[i] = len(inverted_index_dic[vocab_list[i]])

In [21]:
df

array([1., 1., 1., ..., 1., 1., 1.])

In [22]:
df[query_word_0_index], df[query_word_1_index]

(26.0, 15.0)

In [23]:
len(df)

4091

In [24]:
N = 60
np.log(N / df)

array([4.09434456, 4.09434456, 4.09434456, ..., 4.09434456, 4.09434456,
       4.09434456])

In [25]:
# tf-idf를 구한다. 
N = 60
doc_tf_idf = np.log(1 + doc_tf) * np.log(N / df)

In [26]:
doc_tf_idf

array([[0.        , 0.        , 0.        , ..., 2.83798339, 0.        ,
        0.        ],
       [0.        , 0.        , 5.67596678, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

# 7. query의 tf-idf를 구한다.

In [27]:
# query의 tf를 구한다. 
query_tf = np.zeros(len(vocab_set))

query_tf[query_word_0_index] = 1
query_tf[query_word_1_index] = 1

In [28]:
# df는 위의 df를 그대로 사용한다. 
# query_tf_idf를 구한다. 
query_tf_idf = np.log(1 + query_tf) * np.log(N / df)

In [29]:
query_tf_idf

array([0., 0., 0., ..., 0., 0., 0.])

In [30]:
query_tf_idf[query_word_0_index],query_tf_idf[query_word_1_index]

(0.5796429602234836, 0.9609060278364028)

# 8. 각 query와 document간의 cosine similarity를 구한다.

In [31]:
print(doc_tf_idf)

[[0.         0.         0.         ... 2.83798339 0.         0.        ]
 [0.         0.         5.67596678 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [32]:
print(query_tf_idf)

[0. 0. 0. ... 0. 0. 0.]


In [33]:
def cosine_similarity(x,y):
    normalizing_factor_x = np.sqrt(np.sum(np.square(x))) # x의 normalizing factor
    normalizing_factor_y = np.sqrt(np.sum(np.square(y))) # y의 normalizing factor
    
    return np.matmul(x, np.transpose(y)) / (normalizing_factor_x * normalizing_factor_y)

In [34]:
cosine_similarity_list = OrderedDict() # cosine_similarity와 index값을 저장하는 dic

temp = 0

for doc in doc_tf_idf:
    similarity = cosine_similarity(doc, query_tf_idf) #similarity를 구한다. 
    cosine_similarity_list[list(query_doc_set)[temp]] = similarity #cosine_similarity에 index와 similarity를 넣는다.
    temp += 1 # query_doc_set의 index값이다. 

In [35]:
cosine_similarity_list

OrderedDict([(36, 0.06158969974834281),
             (6, 0.04064645671828753),
             (40, 0.06082115922198023),
             (41, 0.04507188836303475),
             (43, 0.0614549900547623),
             (46, 0.10017468119783003),
             (47, 0.07016066770850694),
             (48, 0.08610676376458064),
             (49, 0.06020678998127625),
             (50, 0.07017951617582499),
             (53, 0.04492327826786915),
             (54, 0.13432984784432406),
             (57, 0.03241169289402157),
             (58, 0.0781294135826797)])

# 9. cosine similarity를 기준으로 상위 5개의 document를 구한다.

In [36]:
# cosine_similairty의 값을 기준으로 sort를 한다. 
rank = sorted(cosine_similarity_list.items(), key=lambda x:x[1], reverse=True)

In [37]:
rank

[(54, 0.13432984784432406),
 (46, 0.10017468119783003),
 (48, 0.08610676376458064),
 (58, 0.0781294135826797),
 (50, 0.07017951617582499),
 (47, 0.07016066770850694),
 (36, 0.06158969974834281),
 (43, 0.0614549900547623),
 (40, 0.06082115922198023),
 (49, 0.06020678998127625),
 (41, 0.04507188836303475),
 (53, 0.04492327826786915),
 (6, 0.04064645671828753),
 (57, 0.03241169289402157)]

In [38]:
rank[:5]

[(54, 0.13432984784432406),
 (46, 0.10017468119783003),
 (48, 0.08610676376458064),
 (58, 0.0781294135826797),
 (50, 0.07017951617582499)]

In [39]:
rank_index = []

for tuples in rank:
    rank_index.append(tuples[0])

In [40]:
rank_index

[54, 46, 48, 58, 50, 47, 36, 43, 40, 49, 41, 53, 6, 57]

In [41]:
for index in rank_index:
    print(doc_dictionary_list[index][query_word[0]],doc_dictionary_list[index][query_word[1]])

9 3
2 10
5 4
8 1
3 6
7 10
1 1
1 2
3 1
9 2
15 2
2 3
2 4
4 1


In [42]:
# 상위 5개 document
rank_index[:5]

[54, 46, 48, 58, 50]

# 변형

### 다큐먼트의 tf_idf와 단어의 tf_idf의 차원을 2개로 줄여보자

In [44]:
doc_tf_idf[0][query_word_0_index],doc_tf_idf[0][query_word_1_index]

(0.9187123557612263, 2.2311547025799614)

In [49]:
#2차원 document_tf_idf를 구하자
doc_tf_idf_2 = np.zeros([len(query_doc_set), 2]) # 차원이 2차원인 Vector space를 만들자.

for i in range(len(doc_tf_idf)):
    doc_tf_idf_2[i][0] = doc_tf_idf[i][query_word_0_index]
    doc_tf_idf_2[i][1] = doc_tf_idf[i][query_word_1_index] 

In [50]:
doc_tf_idf_2

array([[0.91871236, 2.2311547 ],
       [0.57964296, 0.96090603],
       [1.15928592, 0.96090603],
       [2.31857184, 1.52300002],
       [0.57964296, 1.52300002],
       [0.91871236, 3.3241887 ],
       [1.73892888, 3.3241887 ],
       [1.49835532, 2.2311547 ],
       [1.92553223, 1.52300002],
       [1.15928592, 2.69760427],
       [0.91871236, 1.92181206],
       [1.92553223, 1.92181206],
       [1.34588927, 0.96090603],
       [1.83742471, 0.96090603]])

In [54]:
# 2차원 term tf_idf를 구하자
query_tf_idf_2 = np.array([query_tf_idf[query_word_0_index], query_tf_idf[query_word_1_index]])

In [57]:
cosine_similarity_list_2 = OrderedDict() # cosine_similarity와 index값을 저장하는 dic

temp = 0

for doc in doc_tf_idf_2:
    similarity = cosine_similarity(doc, query_tf_idf_2) #similarity를 구한다. 
    cosine_similarity_list_2[list(query_doc_set)[temp]] = similarity #cosine_similarity에 index와 similarity를 넣는다.
    temp += 1 # query_doc_set의 index값이다. 

In [58]:
cosine_similarity_list_2

OrderedDict([(36, 0.9884429358620711),
             (6, 0.9999999999999998),
             (40, 0.9441121515765775),
             (41, 0.9018261528455037),
             (43, 0.9840003597567888),
             (46, 0.9629268397836399),
             (47, 0.9981509980413312),
             (48, 0.9988194853219733),
             (49, 0.9363152892955586),
             (50, 0.990643316532008),
             (53, 0.9953129256330802),
             (54, 0.9704814186675252),
             (57, 0.9179252223260421),
             (58, 0.8545256745103544)])

In [59]:
# cosine_similairty의 값을 기준으로 sort를 한다. 
rank_2 = sorted(cosine_similarity_list_2.items(), key=lambda x:x[1], reverse=True)

In [60]:
rank_2

[(6, 0.9999999999999998),
 (48, 0.9988194853219733),
 (47, 0.9981509980413312),
 (53, 0.9953129256330802),
 (50, 0.990643316532008),
 (36, 0.9884429358620711),
 (43, 0.9840003597567888),
 (54, 0.9704814186675252),
 (46, 0.9629268397836399),
 (40, 0.9441121515765775),
 (49, 0.9363152892955586),
 (57, 0.9179252223260421),
 (41, 0.9018261528455037),
 (58, 0.8545256745103544)]

In [61]:
rank_2[:5]

[(6, 0.9999999999999998),
 (48, 0.9988194853219733),
 (47, 0.9981509980413312),
 (53, 0.9953129256330802),
 (50, 0.990643316532008)]

In [62]:
rank_index_2 = []

for tuples in rank_2:
    rank_index_2.append(tuples[0])

In [63]:
rank_index_2

[6, 48, 47, 53, 50, 36, 43, 54, 46, 40, 49, 57, 41, 58]

In [64]:
for index in rank_index_2:
    print(doc_dictionary_list[index][query_word[0]],doc_dictionary_list[index][query_word[1]])

2 4
5 4
7 10
2 3
3 6
1 1
1 2
9 3
2 10
3 1
9 2
4 1
15 2
8 1


In [65]:
# 상위 5개 document
rank_index_2[:5]

[6, 48, 47, 53, 50]